In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [2]:
flights_df = pd.read_csv('Data/File_1.csv')
flights_df = flights_df.drop('Unnamed: 0', axis=1)

C:\Users\dave\AppData\Local\Temp\ipykernel_15348\3269815118.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('Data/File_1.csv')


In [3]:
# Convert 'timestamp' and 'last_position' to datetime
flights_df['timestamp'] = pd.to_datetime(flights_df['timestamp'])
flights_df['last_position'] = pd.to_datetime(flights_df['last_position'])

In [4]:
flights_df.drop_duplicates(subset=['icao24', 'last_position'], inplace = True)
flights_df = flights_df[flights_df['baro_altitude'] >= 10000]
flights_df = flights_df[['timestamp', 'icao24', 'latitude', 'longitude', 'ground_speed', 'track', 'vertical_rate', 'baro_altitude']]

In [5]:
flights_df

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude
929,2022-10-01 17:37:30,4853d1,52.044564,4.795846,295.413015,166.293039,2560.0,10025.0
930,2022-10-01 17:37:31,4853d1,52.043260,4.796317,295.413015,166.293039,2688.0,10050.0
931,2022-10-01 17:37:32,4853d1,52.041910,4.796866,295.651567,166.104763,2752.0,10100.0
932,2022-10-01 17:37:33,4853d1,52.040467,4.797494,296.863349,165.963757,2816.0,10150.0
933,2022-10-01 17:37:34,4853d1,52.039117,4.798043,297.107468,165.776669,2752.0,10200.0
...,...,...,...,...,...,...,...,...
1082899,2022-10-30 21:55:02,484133,52.339519,4.709957,141.226646,183.247343,0.0,40000.0
1082983,2022-10-30 21:56:26,484133,52.330330,4.716034,141.226646,183.247343,0.0,40000.0
1082999,2022-10-30 21:56:42,484133,52.329014,4.718380,141.226646,183.247343,0.0,40000.0
1083022,2022-10-30 21:57:05,484133,52.328419,4.721928,141.226646,183.247343,0.0,18300.0


In [6]:
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     3
 track            3
 vertical_rate    3
 baro_altitude    0
 dtype: int64,
 (731097, 8))

In [7]:
flights_df.dropna(inplace = True)
flights_df.isnull().sum(), flights_df.shape

(timestamp        0
 icao24           0
 latitude         0
 longitude        0
 ground_speed     0
 track            0
 vertical_rate    0
 baro_altitude    0
 dtype: int64,
 (731094, 8))

In [8]:
flights_df_sorted = flights_df.sort_values(by=['icao24', 'timestamp'])
flights_df_sorted

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude
1081344,2022-10-30 21:29:07,484133,50.280304,2.691000,524.069199,32.295105,0.0,34000.0
1081345,2022-10-30 21:29:08,484133,50.282329,2.692987,524.069199,32.295105,0.0,34000.0
1081346,2022-10-30 21:29:09,484133,50.284744,2.694903,524.069199,32.295105,0.0,34000.0
1081347,2022-10-30 21:29:10,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0
1081348,2022-10-30 21:29:11,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0
...,...,...,...,...,...,...,...,...
625047,2022-10-29 22:28:39,ad5b90,38.817169,-75.942874,288.949573,297.629247,-1664.0,10100.0
625048,2022-10-29 22:28:40,ad5b90,38.817537,-75.943726,288.999750,298.072487,-1664.0,10075.0
625049,2022-10-29 22:28:41,ad5b90,38.818405,-75.945799,288.591157,298.341095,-1600.0,10050.0
625050,2022-10-29 22:28:42,ad5b90,38.819027,-75.947327,288.188925,298.610460,-1600.0,10025.0


In [9]:
# Function to find the row with the closest timestamp to the target time
def find_first_future_row(df, target_time, icao24):
    future_rows = df[(df['icao24'] == icao24) & (df['timestamp'] >= target_time) & (df['timestamp'] <= target_time + timedelta(minutes=10))]
    return future_rows.iloc[0] if not future_rows.empty else None

# Testing the function on a small subset of the data
subset_df = flights_df_sorted.head(1000)

# Adding new columns for future position and altitude
subset_df['latitude_in_10min'] = np.nan
subset_df['longitude_in_10min'] = np.nan
subset_df['baro_altitude_in_10min'] = np.nan

# Iterating over each row in the subset
for index, row in subset_df.iterrows():
    future_time = row['timestamp'] + timedelta(minutes=10)
    future_row = find_first_future_row(subset_df, future_time, row['icao24'])
    if future_row is not None:
        subset_df.at[index, 'latitude_in_10min'] = future_row['latitude']
        subset_df.at[index, 'longitude_in_10min'] = future_row['longitude']
        subset_df.at[index, 'baro_altitude_in_10min'] = future_row['baro_altitude']

# Displaying the modified subset to check the results
subset_df.head()
subset_df.isna().sum()

C:\Users\dave\AppData\Local\Temp\ipykernel_15348\3155901338.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['latitude_in_10min'] = np.nan
C:\Users\dave\AppData\Local\Temp\ipykernel_15348\3155901338.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['longitude_in_10min'] = np.nan
C:\Users\dave\AppData\Local\Temp\ipykernel_15348\3155901338.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

timestamp                   0
icao24                      0
latitude                    0
longitude                   0
ground_speed                0
track                       0
vertical_rate               0
baro_altitude               0
latitude_in_10min         127
longitude_in_10min        127
baro_altitude_in_10min    127
dtype: int64

In [10]:
subset_df

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,baro_altitude,latitude_in_10min,longitude_in_10min,baro_altitude_in_10min
1081344,2022-10-30 21:29:07,484133,50.280304,2.691000,524.069199,32.295105,0.0,34000.0,51.472183,3.798597,17775.0
1081345,2022-10-30 21:29:08,484133,50.282329,2.692987,524.069199,32.295105,0.0,34000.0,51.474066,3.799973,17750.0
1081346,2022-10-30 21:29:09,484133,50.284744,2.694903,524.069199,32.295105,0.0,34000.0,51.475891,3.801344,17700.0
1081347,2022-10-30 21:29:10,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0,51.477651,3.802872,17675.0
1081348,2022-10-30 21:29:11,484133,50.286705,2.697069,524.069199,32.295105,0.0,34000.0,51.479416,3.804165,17625.0
...,...,...,...,...,...,...,...,...,...,...,...
356292,2022-10-09 17:38:47,484130,41.407378,1.880259,359.201583,346.311937,3072.0,13500.0,NaN,NaN,NaN
356293,2022-10-09 17:38:48,484130,41.409943,1.879723,359.201583,346.311937,3072.0,13575.0,NaN,NaN,NaN
356294,2022-10-09 17:38:49,484130,41.410636,1.879237,359.201583,346.311937,3008.0,13600.0,NaN,NaN,NaN
356295,2022-10-09 17:38:50,484130,41.412277,1.878662,359.201583,346.311937,2944.0,13650.0,NaN,NaN,NaN
